In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv")
ss = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows', 100)

In [ ]:
print(f'Shape of Train dataset is : {train.shape}')
print(f'Shape of Test dataset is : {test.shape}')
print(f'Shape of Sample Submission dataset is : {ss.shape}')

In [ ]:
def check_NAN_columns(df, df_name):
    if len(df.columns[df.isnull().any()]) == 0:
        print(f'No missing data in {df_name} dataset')
    else:
        print(f'The following columns are having missing data in {df_name} dataset:')
        print(df.columns[df.isnull().any()])
        
datasets = {
    'Train': train,
    'Test': test,
    'Sample Submission': ss,
}
for df_name, df in datasets.items():
    check_NAN_columns(df, df_name)

In [ ]:
train.drop(columns=['id']).describe().T


In [ ]:
test.drop(columns=['id']).describe().T

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.countplot(x='target', data=train)
ax.set_title('Target Distribution')

In [ ]:
X = train.drop(["id","target"], axis=1)
y = train.target
X_test = test.drop("id", axis=1)

In [ ]:
iterations = 100
lgbm_pred = 0
logloss = []
for i in tqdm_notebook(range(iterations)):
    random_state = np.random.randint(0,2000)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y , test_size=0.2, stratify=y, random_state=random_state)
    lgbm = LGBMClassifier(random_state=random_state)
    lgbm.fit(X_train,y_train)
    valid_preds = lgbm.predict_proba(X_valid)
    lgbm_pred += lgbm.predict_proba(X_test)/iterations
    logloss.append(log_loss(y_valid,valid_preds))
    print(f'Iteration {i} : Random State = {random_state}, Log Loss = {log_loss(y_valid,valid_preds)}')
print(f'Average Log Loss = {sum(logloss)/len(logloss)}')

In [ ]:
ss["Class_1"] = lgbm_pred[:,0]
ss["Class_2"] = lgbm_pred[:,1]
ss["Class_3"] = lgbm_pred[:,2]
ss["Class_4"] = lgbm_pred[:,3]
ss["Class_5"] = lgbm_pred[:,4]
ss["Class_6"] = lgbm_pred[:,5]
ss["Class_7"] = lgbm_pred[:,6]
ss["Class_8"] = lgbm_pred[:,7]
ss["Class_9"] = lgbm_pred[:,8]
ss.to_csv("/kaggle/working/lgbm_100_ensemble_sub.csv", index=False)